In [1]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, AutoModelForSeq2SeqLM, GenerationConfig

/static/user/spock/.cache/pypoetry/virtualenvs/ai-iwSNfyWa-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [84]:
toxicity_model_name = "facebook/roberta-hate-speech-dynabench-r4-target"
# toxicity_model_name = "unitary/toxic-bert"
# toxicity_tokenizer = AutoTokenizer.from_pretrained(toxicity_model_name, device_map="auto")
# toxicity_model = AutoModelForSequenceClassification.from_pretrained(toxicity_model_name, device_map="auto")
toxicity_tokenizer = AutoTokenizer.from_pretrained(toxicity_model_name)
toxicity_model = AutoModelForSequenceClassification.from_pretrained(toxicity_model_name)

print(toxicity_model.config.id2label)

{0: 'nothate', 1: 'hate'}


In [127]:
toxicity_model.device

device(type='cpu')

In [128]:
ids = toxicity_tokenizer('fuck you ', return_tensors="pt").input_ids

In [129]:
# toxicity_model

In [130]:
_=toxicity_model.eval()

In [132]:
toxicity_model(input_ids=ids).logits

tensor([[-3.4335,  3.0830]], grad_fn=<AddmmBackward0>)